In [1]:
import pandas as pd
from pathlib import Path
import torch
import transformers
import random
import transformers
from nltk import pos_tag
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Antonio\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
sns.set_theme();

In [3]:
RESOURCES_DIR = Path("../resources")
DATASETS_PATH = RESOURCES_DIR / "datasets"
WORD_EMBEDDINGS_NAME = "glove.42B.300d"
DUMPS_DIR = RESOURCES_DIR / "DUMPS"

In [4]:
SIMPLETEXT_DATASET_PATH = DATASETS_PATH / "simpleText/task 3"
SIMPLETEXT_TRAIN_PATH = SIMPLETEXT_DATASET_PATH / "train"

In [5]:
complex_text = pd.read_csv(SIMPLETEXT_TRAIN_PATH / "simpleText.train.complex.txt", header=None, sep="\t", names=["original_text"])
simple_text = pd.read_csv(SIMPLETEXT_TRAIN_PATH / "simpleText.train.simple.txt", header=None, sep="\t",names=["simple_text"])

In [6]:
sentences_pairs = pd.concat([complex_text, simple_text], axis=1)
sentences_pairs

,original_text,simple_text
0,"In the modern era of automation and robotics, ...",Current academic and industrial research is in...
1,With the ever increasing number of unmanned ae...,Drones are increasingly used in the civilian a...
2,Due to guidelines set by the governments regar...,Governments set guidelines on the operation ce...
3,In an attempt to achieve the above mentioned t...,Researchers propose data-driven solutions allo...
4,Derived from the classic image classification ...,"The algorithm, based on the Inception model, d..."
...,...,...
643,Bodybuilders generally train with moderate loa...,Bodybuilders train with moderate loads and sho...
644,"Powerlifters, on the other hand, routinely tra...","Powerlifters, on the other hand, train with hi..."
645,Although both groups are known to display impr...,Although both groups are known to display impr...
646,It has been shown that many factors mediate th...,It has been shown that many factors mediate th...


In [7]:
model = transformers.AutoModelWithLMHead.from_pretrained('lordtt13/COVID-SciBERT')
tokenizer = transformers.AutoTokenizer.from_pretrained('lordtt13/COVID-SciBERT')

c:\users\antonio\pycharmprojects\simpletextclef\venv\lib\site-packages\transformers\models\auto\modeling_auto.py:742: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [8]:
def mask_sentence_prediction(text, topk=50):
    #print("----------------------")
    sentence_tokens = [token[0] for token in tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)]
    pos_tagging_sentence = pos_tag(sentence_tokens)
    #print(pos_tag(sentence_tokens))
    #print(len(pos_tagging_sentence))
    predictions = []
    for i, (word, pos) in enumerate(pos_tagging_sentence):

        #print(f"i:{i}, word:{word}, pos:{pos}")
        if pos in ['NNS', 'NN', 'VBP', 'VBG', 'VBD']:
            sentence_masked = sentence_tokens.copy()
            sentence_masked[i] = "[MASK]"
            sentence_final = " ".join(sentence_masked)
            #print(sentence_final)
            inputs = tokenizer(sentence_final, return_tensors="pt")
            token_logits = model(**inputs).logits
            mask_token_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]
            mask_token_logits = token_logits[0,mask_token_index,:]
            top_n_tokens = torch.topk(mask_token_logits,topk,dim=1).indices[0].tolist()
            predicted_tokens = [tokenizer.decode([token]) for token in top_n_tokens]
            if word in predicted_tokens:
                predictions.append(predicted_tokens.index(word))


    predictions = [np.log(pred+1) for pred in predictions]
    #print(predictions)
    if len(predictions) == 0:
            return np.log(1 + topk*50)

    value = np.quantile(predictions, 0.75)
    return value

In [9]:
#mask_sentence_prediction("The computational efficiency of MAVNet enables the drone to fly at high speeds of up to 6m/sec.")

In [10]:
def check_pair_sentence(complex, simple):
    complex_feature = mask_sentence_prediction(complex)
    simple_feature = mask_sentence_prediction(simple)
    #print(f"Complex feature: {complex_feature}. Simple feature: {simple_feature}")
    if complex_feature !=0:
        value = simple_feature/complex_feature
    else:
        value = 1
    if value > 5:
        print(f"Complex feature: {complex_feature}. Simple feature: {simple_feature}")
    return value
    #return min(value,2)

In [11]:
complex = "The computational efficiency of MAVNet enables the drone to fly at high speeds of up to 6m/sec."
simple = "MAVNet computational efficiency enables the drone to fly up to 6m/sec."

In [12]:
check_pair_sentence(complex, simple)

0.9866183394175266

In [ ]:
%time
results = []
for i,row in sentences_pairs.iterrows():
    value = check_pair_sentence(row['original_text'], row['simple_text'])
    results.append(value)


print("Finish")

CPU times: total: 0 ns
Wall time: 0 ns
Complex feature: 1.3862943611198906. Simple feature: 7.824445930877619
Complex feature: 1.3438196019210413. Simple feature: 7.824445930877619
Complex feature: 0.27465307216702745. Simple feature: 7.824445930877619
Complex feature: 0.6931471805599453. Simple feature: 7.824445930877619
Complex feature: 0.34657359027997264. Simple feature: 2.727660557678851
Complex feature: 0.34657359027997264. Simple feature: 2.668769039850659


In [ ]:
results

In [ ]:
fig = plt.figure(figsize=(10,6))
x = pd.Series(results)
ax = sns.distplot(x,label="LM", bins=[0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2], kde=False)
fig.legend(labels=['LM'])
ax.set(xlabel='LMFMR values', ylabel='Training Data Documents')
plt.show()